<a href="https://colab.research.google.com/github/ankitdandiwala/ANN/blob/main/Obejctive4_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn import metrics


In [2]:
#Read data
dataset = pd.read_excel("Dataset.xlsx")

#Identifiying input and output features as separate variable
X = dataset.iloc[:, 0:2].values
y = dataset.iloc[:, 5:7].values

#Inout feeature correspondinng to Double Lane Change manaoeuvre
DLC_df = pd.read_excel("DLCat30kmph.xlsx")
x_DLC = DLC_df.iloc[:,1:3].values

# Splitting the dataset into the Training set and Test set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 2)

# Feature Scaling (Using Min-max)
from sklearn.preprocessing import MinMaxScaler #(feature_range=(0, 1), *, copy=True, clip=False)
minmax = MinMaxScaler(feature_range=(0, 1))
X_train = minmax.fit_transform(X_train)
X_val = minmax.transform(X_val)
x_DLC = minmax.transform(x_DLC)

# Initialising the ANN
model = Sequential()
EPOCH = 50

# Adding the input layer and the first hidden layer 
model.add(Dense(4, activation = 'relu', input_dim = 2))
#model.add(Dense(4, activation = 'relu', input_dim = 2, kernel_initializer=initializers.RandomNormal(stddev=0.01),
#    bias_initializer=initializers.Zeros()))

# Adding the second hidden layer
model.add(Dense(units = 4, activation = 'relu'))
#model.add(Dense(units = 4, activation = 'relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
#    bias_regularizer=regularizers.L2(1e-4),
#    activity_regularizer=regularizers.L2(1e-5)))

# Adding the third hidden layer
model.add(Dense(units = 4, activation = 'relu'))

# Adding the output layer
model.add(Dense(units = 2))

# Compiling the ANN
model.compile(optimizer = keras.optimizers.Adam(decay = 1e-5), loss = 'mean_squared_error', metrics = ['accuracy'])

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train, batch_size = 16, epochs = EPOCH, validation_data=(X_val, y_val))

#Predicted output
y_pred = model.predict(X_val)
y_pred_DLC = model.predict(x_DLC)

#Plot real vs predicted data
plt.plot( sorted(y_val[:,0]), color = 'red', label = 'Real data')
plt.plot( sorted(y_pred[:,0]), color = 'blue', label = 'Predicted data')
plt.legend()
plt.show()
plt.plot( sorted(y_val[:,1]), color = 'red', label = 'Real data')
plt.plot( sorted(y_pred[:,1]), color = 'blue', label = 'Predicted data')
plt.legend()
plt.show()

#Variables for plot
loss_train = history.history['loss']
loss_test = history.history['val_loss']
accuracy_train = history.history['accuracy']
accuracy_test = history.history['val_accuracy']
epochs_x = [i for i in range(EPOCH)]

#Plots of all accuracy and loss
plt.plot(epochs_x, loss_train, 'g', label='Training loss')
plt.plot(epochs_x, loss_test, 'b', label='Test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(epochs_x, accuracy_train, 'y', label='Training accuracy')
plt.plot(epochs_x, accuracy_test, 'r', label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#Printing Results
print("Error in tilt angle [MAE, MSE, RMSE, RS]:")
#'Mean Absolute Error-TiltAngle:'
print(metrics.mean_absolute_error(y_val[:,0], y_pred[:,0]))
#'Mean Squared Error-Tilt angle:'
print(metrics.mean_squared_error(y_val[:,0], y_pred[:,0]))
#'Root Mean Squared Error-Tila angle:'
print(np.sqrt(metrics.mean_squared_error(y_val[:,0], y_pred[:,0])))
#'R square-Tila angle:'
print(metrics.r2_score(y_val[:,0], y_pred[:,0]))
print(print("Error in cs angle [MAE, MSE, RMSE, RS]:"))
#'Mean Absolute Error-csAngle:'
print(metrics.mean_absolute_error(y_val[:,1], y_pred[:,1]))
#'Mean Squared Error-csAngle:'
print(metrics.mean_squared_error(y_val[:,1], y_pred[:,1]))
#'Root Mean Squared Error-csAngle:'
print(np.sqrt(metrics.mean_squared_error(y_val[:,1], y_pred[:,1])))
#'R square-csAngle:'
print(metrics.r2_score(y_val[:,1], y_pred[:,1]))
print("Error in loss and accuracy")
print('Mean training accuracy:',np.mean(accuracy_train))
print('Mean test accuracy:',np.mean(accuracy_test))
print('Mean training loss:',np.mean(loss_train))
print('Mean test loss:',np.mean(loss_test))



#Export Data
result_Accu_Loss = pd.DataFrame(history.history)
result_Accu_Loss.to_csv('result_Accu_Loss_4.csv')
result_pred = pd.DataFrame(data = y_pred, columns = ['ata_pred', 'csAngle_pred'])
result_test = pd.DataFrame(data = y_val, columns = ['ata_test', 'csAngle_test'])
result_test_pred = pd.concat([result_pred,result_test], axis = 1)
result_test_pred.to_csv('result_test_pred_4.csv')
print(result_Accu_Loss.iloc[-1,:])
result_pred_DLC = pd.DataFrame(data = y_pred_DLC, columns = ['ata_pred', 'csAngle_pred'])
result_pred_DLC.to_csv('result_pred_DLC_3.csv')

#Exporting model parameter in .h5 format to view the same on netron
model.save("saved_model_2442.h5") # Visualize h5 on https://netron.app/ 

FileNotFoundError: ignored